In [1]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import numpy as np

PIPELINE_ROOT = Path('../src').resolve().parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())
print(PIPELINE_ROOT)

from library.registration.brain_structure_manager import BrainStructureManager
from library.registration.algorithm import umeyama

/home/eddyod/programming/preprocessing-pipeline/src


In [2]:
def brain_to_atlas_transform(brain_coord, r, t):
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    atlas_coord = r @ brain_coord + t
    return atlas_coord.T[0] # Convert back to a row vector

def calculate_distance(com1, com2):
    return (np.linalg.norm(com1 - com2))
        
def plot_point_sets_3d(point_sets):
    df = pd.DataFrame()
    for data, label in point_sets:
        df_cur = pd.DataFrame(data.T, columns=['x', 'y', 'z'])
        df_cur['label'] = label
        #df = df.concat(df_cur, ignore_index=True)
        df = pd.concat([df, df_cur], axis= 0)
    
    fig = px.scatter_3d(df, x='x', y='y', z='z', color='label', width=800, height=600)
    fig.update_layout(
    scene={
        'xaxis': {'range': (0, 528)}, 
        'yaxis': {'range': (0, 320)},
        'zaxis': {'range': (0, 456)},
    })
    return fig
        

In [3]:
animal = 'Atlas'
brain = BrainStructureManager(animal)
brain.fixed_brain = BrainStructureManager('Allen')

In [ ]:
brainstem_keys = set(brain.allen_structures_keys) - brain.midbrain_keys 

In [4]:
moving_coms = brain.get_coms(annotator_id=1)
allen_coms = brain.fixed_brain.get_coms(annotator_id=1)
common_keys = sorted(allen_coms.keys() & moving_coms.keys())
allen_points = np.array([allen_coms[s] for s in common_keys])
moving_points = np.array([moving_coms[s] for s in common_keys])

In [5]:
common_midbrain_keys = sorted(allen_coms.keys() & moving_coms.keys() & brain.midbrain_keys)
allen_midbrain_points = np.array([allen_coms[s] for s in common_midbrain_keys])
moving_midbrain_points = np.array([moving_coms[s] for s in common_midbrain_keys])

In [6]:
allen_point_dict = {s:allen_coms[s] for s in common_keys}
moving_point_dict = {s:moving_coms[s] for s in common_keys}

In [8]:
len(allen_point_dict)

37

In [ ]:
distances = []
sortme = {}
for structure in common_keys:
    (x,y,z) = allen_point_dict[structure]
    allen_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    #print(atlas_point, allen_point)
    d = calculate_distance(allen_point, moving_point)
    distances.append(d)
    sortme[structure] = d
print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
for structure, d in ds.items():
    print(f'{structure} distance from Allen={round(d,2)} micrometers')

In [ ]:
plot_point_sets_3d([
    (moving_points.T/25, 'unaligned moving centers'),
    (allen_points.T/25, 'Allen centers')
])

In [ ]:
(RMSD, R1, t1, _) = Superpose3D(allen_points, moving_points, allow_rescale=True)
t1 = t1.reshape(3,1)
print(RMSD)

In [ ]:
allen_midbrain_points.shape

In [ ]:
R, t = umeyama(moving_points.T, allen_points.T)

In [ ]:
t

In [ ]:
reg_points = R @ moving_points.T + t

 That does not preclude starting the new idea but is just to say that somehow we need to get David's attention on what you have done- do a graph
= box plot of the midbrain com distances vs allen (active atlas based)
=box plot of the brainstem vs allen distances (but not using midbrain values to improve the distances)
This should provide an easy visual...


When the tethering is done for the brainstem structures though- he does not want the midbrain metrics to be included ( I don't know though if he means that the alignment is done with midbrain and then the evaluated outputs are only for the brainstem structures...?) your guess is best
So- in any case it was clear that he did not want metrics for midbrain and brainstem structures to be combined.

In [ ]:
print('Mean')
print(np.mean(allen_points, axis=0))
print(np.mean(reg_points, axis=1))
print('Min')
print(np.min(allen_points, axis=0))
print(np.min(reg_points, axis=1))
print('Max')
print(np.max(allen_points, axis=0))
print(np.max(reg_points, axis=1))

In [ ]:
reg_point_dict = {s:moving_coms[s] for s in common_keys}

In [ ]:
distances = []
distances2 = []
sortme = {}
for structure in common_keys:
    (x,y,z) = allen_point_dict[structure]
    sitk_point = reg_dict[structure]
    allen_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    reg_point = brain_to_atlas_transform(moving_point, R, t)
    d = calculate_distance(allen_point, reg_point)
    ds = calculate_distance(allen_point, sitk_point)
    distances2.append(ds)
    distances.append(d)
    print(f'{structure} COM={reg_point.round()} SITK={round(ds,2)} distance={round(d,2)}')
    sortme[structure] = d

print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
print(f'n={len(distances2)}, min={min(distances2)} max={max(distances2)}, mean={np.mean(distances2)}')
#ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
#for structure, d in ds.items():
#    print(f'{structure} {round(d,2)}')

In [ ]:
reg_points.shape

In [ ]:
plot_point_sets_3d([
    (reg_points/25, 'registered moving centers'),
    (allen_points.T/25, 'Allen centers')    
])